In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold

from scipy.spatial.distance import pdist, squareform
from minepy import MINE
from sklearn.preprocessing import StandardScaler
from scipy import stats
import scipy

from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings("ignore")

In [2]:
# 1_remove_low_variance_features
file_path ='../Data01-descriptor_Statisticalselection'
file='11all_data.pkl'
all_data = pd.read_pickle(file_path+file)

X = all_data.iloc[:,2:324]
y = all_data.iloc[:,0:2]

X_var = pd.DataFrame(X.var())

vt = VarianceThreshold(threshold = 0.01)
X_selected = vt.fit_transform(X)
lowvariance_data = pd.DataFrame(X_selected)

all_name = X.columns.values.tolist()
select_name_index0 = vt.get_support(indices=True)
select_name0 = []
for i in select_name_index0:
    select_name0.append(all_name[i])

lowvariance_data.columns = select_name0

file1 = r"11all_data_remove_low_variance.pkl"
lowvariance_data_y = pd.concat((y,lowvariance_data),axis = 1)
lowvariance_data_y.to_pickle(file_path+file1)

In [3]:
# 2_correlation_screening_features
file2 = r'12_all_data_remove_low_variance.pkl'
all_data = pd.read_pickle(file_path+file2)
data = all_data.iloc[:,all_data.columns != "ID"]
descriptor_data = data.iloc[:,data.columns != "Kavg_log2"]
all_data_name_list = list(all_data)
descriptor_name_list = list(descriptor_data)
descriptor_count = len(descriptor_name_list)

scaler = StandardScaler()
data_scaler = scaler.fit_transform(data)
DataFrame_data_scaler = pd.DataFrame(data_scaler)

data_pearson = DataFrame_data_scaler.corr(method = 'pearson')
data_spearman = DataFrame_data_scaler.corr(method = 'spearman')
mine = MINE(alpha=0.6, c=15)


def distcorr(X, Y):
    X = np.atleast_1d(X)
    Y = np.atleast_1d(Y)
    if np.prod(X.shape) == len(X):
        X = X[:, None]
    if np.prod(Y.shape) == len(Y):
        Y = Y[:, None]
    X = np.atleast_2d(X)
    Y = np.atleast_2d(Y)
    n = X.shape[0]
    if Y.shape[0] != X.shape[0]:
        raise ValueError('Number of samples must match')
    a = squareform(pdist(X))
    b = squareform(pdist(Y))
    A = a - a.mean(axis=0)[None, :] - a.mean(axis=1)[:, None] + a.mean()
    B = b - b.mean(axis=0)[None, :] - b.mean(axis=1)[:, None] + b.mean()

    dcov2_xy = (A * B).sum() / float(n * n)
    dcov2_xx = (A * A).sum() / float(n * n)
    dcov2_yy = (B * B).sum() / float(n * n)
    dcor = np.sqrt(dcov2_xy) / np.sqrt(np.sqrt(dcov2_xx) * np.sqrt(dcov2_yy))
    return dcor

column_descriptor = data_scaler.shape[1]

Threshold = 0.05
pearson_correlation_list = []
pearson_pvalue_list = []
pearson_selection_list = []
pearson_list = []
for i in range(1,column_descriptor):
    pearson_correlation_list.append(scipy.stats.pearsonr(data_scaler[:,i],data_scaler[:,0])[0])
    pearson_pvalue_list.append(scipy.stats.pearsonr(data_scaler[:,i],data_scaler[:,0])[1])
    if pearson_pvalue_list[i-1] > Threshold:
        pearson_selection_list.append(0)
    else :
        pearson_selection_list.append(1)
pearson_list.append(pearson_correlation_list)
pearson_list.append(pearson_pvalue_list)
pearson_list.append(pearson_selection_list)

Threshold = 0.05
spearman_correlation_list = []
spearman_pvalue_list = []
spearman_selection_list = []
spearman_list = []
for i in range(1,column_descriptor):
    spearman_correlation_list.append(scipy.stats.spearmanr(data_scaler[:,i],data_scaler[:,0])[0])
    spearman_pvalue_list.append(scipy.stats.spearmanr(data_scaler[:,i],data_scaler[:,0])[1])
    if spearman_pvalue_list[i-1] > Threshold:
        spearman_selection_list.append(0)
    else :
        spearman_selection_list.append(1)
spearman_list.append(spearman_correlation_list)
spearman_list.append(spearman_pvalue_list)
spearman_list.append(spearman_selection_list)

Threshold = 0.153
distance_correlation_list = []
distance_selection_list = []
distance_list = []
for i in range(1,column_descriptor):
    distance_correlation_list.append(distcorr(data_scaler[:,i],data_scaler[:,0]))
    if abs(distance_correlation_list[i-1]) <= Threshold:
        distance_selection_list.append(0)
    else :
        distance_selection_list.append(1)
distance_list.append(distance_correlation_list)
distance_list.append(distance_selection_list)

Threshold = 0.132
mic_correlation_list = []
mic_selection_list = []
mic_list = []
for i in range(1,column_descriptor):
    mine.compute_score(data_scaler[:,i],data_scaler[:,0])
    mic_correlation_list.append(mine.mic())
    if abs(mic_correlation_list[i-1]) <= Threshold:
        mic_selection_list.append(0)
    else:
        mic_selection_list.append(1)
mic_list.append(mic_correlation_list)
mic_list.append(mic_selection_list)

sum_list = []
for j in range(0,descriptor_count):
    sum_list.append(pearson_selection_list[j] + spearman_selection_list[j] + distance_selection_list[j] + mic_selection_list[j])

sum_selection_list1 = []
Threshold1 = 1
for j in range(0,descriptor_count):
    if sum_list[j] >= Threshold1:
        sum_selection_list1.append(1)
    else:
        sum_selection_list1.append(0)
sum(sum_selection_list1)

sum_selection_list2 = []
Threshold2 = 2
for j in range(0,descriptor_count):
    if sum_list[j] >= Threshold2:
        sum_selection_list2.append(1)
    else:
        sum_selection_list2.append(0)
sum(sum_selection_list2)

sum_selection_list3 = []
Threshold3 = 3
for j in range(0,descriptor_count):
    if sum_list[j] >= Threshold3:
        sum_selection_list3.append(1)
    else:
        sum_selection_list3.append(0)
sum(sum_selection_list3)

sum_selection_list4 = []
Threshold4 = 4
for j in range(0,descriptor_count):
    if sum_list[j] >= Threshold4:
        sum_selection_list4.append(1)
    else:
        sum_selection_list4.append(0)
sum(sum_selection_list4)

sum_list_all = []
sum_list_all.append(sum_selection_list1)
sum_list_all.append(sum_selection_list2)
sum_list_all.append(sum_selection_list3)
sum_list_all.append(sum_selection_list4)
sum_list_all.append(sum_list)
selection_list_all = []
selection_list_all.append(descriptor_name_list)
selection_list_all.append(pearson_list[0])
selection_list_all.append(pearson_list[1])
selection_list_all.append(pearson_list[2])
selection_list_all.append(spearman_list[0])
selection_list_all.append(spearman_list[1])
selection_list_all.append(spearman_list[2])
selection_list_all.append(distance_list[0])
selection_list_all.append(distance_list[1])
selection_list_all.append(mic_list[0])
selection_list_all.append(mic_list[1])
selection_list_all.append(sum_list_all[0])
selection_list_all.append(sum_list_all[1])
selection_list_all.append(sum_list_all[2])
selection_list_all.append(sum_list_all[3])
selection_list_all.append(sum_list_all[4])
selection_list_all = pd.DataFrame(selection_list_all)

selection_list_all_transpose = selection_list_all.T
selection_list_all_transpose.rename(columns={0:'descriptor_name',1:'pearson_correlation',2:'pearson_pvalue',3:'pearson_selection',
                                             4:'spearman_correlation',5:'spearman_pvalue',6:'spearman_selection',7:'distance_correlation',
                                             8:'distance_selection',9:'mic_correlation',10:'mic_selection',11:'sum_1',
                                             12:'sum_2',13:'sum_3',14:'sum_4',15:'sum'},inplace=True)

filter_data1 = all_data
for k in range(0,len(sum_selection_list1)):
    if sum_selection_list1[k] == 0:
        filter_data1 = filter_data1.drop(descriptor_name_list[k],axis=1)

filter_data2 = all_data
for k in range(0,len(sum_selection_list2)):
    if sum_selection_list2[k] == 0:
        filter_data2 = filter_data2.drop(descriptor_name_list[k],axis=1)

filter_data3 = all_data
for k in range(0,len(sum_selection_list3)):
    if sum_selection_list3[k] == 0:
        filter_data3 = filter_data3.drop(descriptor_name_list[k],axis=1)

filter_data4 = all_data
for k in range(0,len(sum_selection_list4)):
    if sum_selection_list4[k] == 0:
        filter_data4 = filter_data4.drop(descriptor_name_list[k],axis=1)
        
selection_list_all_transpose = selection_list_all_transpose.set_index('descriptor_name')
filter_data1 = filter_data1.set_index('ID')
filter_data2 = filter_data2.set_index('ID')
filter_data3 = filter_data3.set_index('ID')
filter_data4 = filter_data4.set_index('ID')
selection_list_all_transpose.to_pickle(file_path+'13_correlation_screening_statistical_results.pkl')
filter_data1.to_pickle(file_path+'14_filter_threshold_1.pkl')
filter_data2.to_pickle(file_path+'15_filter_threshold_2.pkl')
filter_data3.to_pickle(file_path+'16_filter_threshold_3.pkl')
filter_data4.to_pickle(file_path+'17_filter_threshold_4.pkl')

In [4]:
# 3_mlxtend_screening_features
# This part is implemented through repeated iterations, which requires a long calculation time. Here we only show the running demo.
# The number of iterations corresponds to statistics. This paper carried out a total of 100 iterations, and finally screened the descriptors.
data = pd.read_pickle(file_path+'17_filter_threshold_4.pkl')

X = data.iloc[:,1:54]
y = data.iloc[:,1]
y_array = np.array(y)

# Calculation parameter settings 
max_feature_number = 5
selection_iteration = 5

rfr = RandomForestRegressor()
csv_data = []
csv_data_one_col = []

for selection_iteration_i in range(0,selection_iteration):
    print("The current iteration count is",selection_iteration_i+1)
    X = shuffle(X.T).T
    X_array = np.array(X)
    std = StandardScaler()
    X_array_std = std.fit_transform(X_array)
    sfs = SFS(estimator=rfr,k_features=max_feature_number,forward=True,floating=True,verbose=2,scoring='r2',cv=5)
    sfs.fit(X_array_std,y_array)
    log_dict = sfs.get_metric_dict()
    avg_score_list = []
    for max_feature_i in range(1,max_feature_number+1):
        avg_score_list.append(log_dict[max_feature_i]['avg_score'])
    max_avg_score = max(avg_score_list)
    max_score_feature_number = avg_score_list.index(max_avg_score)+1
    feature_name_list = []
    for feature_name_i in range (0,max_score_feature_number):
        feature_name_list.append(X.columns.values.tolist()[int(log_dict[max_score_feature_number]['feature_names'][feature_name_i])-1])
        csv_data_one_col.append(X.columns.values.tolist()[int(log_dict[max_score_feature_number]['feature_names'][feature_name_i])-1])
    csv_data.append(feature_name_list)

csv_data = pd.DataFrame(csv_data)

file_demo = '18_descriptor_mlxtend_screening_result.pkl'
csv_data.to_pickle(file_path+file_demo)

The current iteration count is 1


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  53 out of  53 | elapsed:   35.8s finished

[2022-11-23 15:17:53] Features: 1/5 -- score: 0.9996304520173028[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  52 out of  52 | elapsed:   45.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s finished

[2022-11-23 15:18:40] Features: 2/5 -- score: 0.9996414728155274[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs

The current iteration count is 2


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  53 out of  53 | elapsed:   33.5s finished

[2022-11-23 15:22:15] Features: 1/5 -- score: 0.9996433768800088[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  52 out of  52 | elapsed:   44.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s finished

[2022-11-23 15:23:00] Features: 2/5 -- score: 0.9996553175953748[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  51 out of  51 | elapsed:   50.1s finished
[Parallel(n_jobs=1)]: Using

The current iteration count is 3


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  53 out of  53 | elapsed:   33.7s finished

[2022-11-23 15:27:22] Features: 1/5 -- score: 0.9996288310637116[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  52 out of  52 | elapsed:   45.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s finished

[2022-11-23 15:28:07] Features: 2/5 -- score: 0.9996532283357331[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  51 out of  51 | elapsed:   48.0s finished
[Parallel(n_jobs=1)]: Using

The current iteration count is 4


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  53 out of  53 | elapsed:   33.8s finished

[2022-11-23 15:31:40] Features: 1/5 -- score: 0.9996412915849306[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  52 out of  52 | elapsed:   44.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s finished

[2022-11-23 15:32:25] Features: 2/5 -- score: 0.9996477816661006[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  51 out of  51 | elapsed:   45.9s finished
[Parallel(n_jobs=1)]: Using

The current iteration count is 5


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  53 out of  53 | elapsed:   34.9s finished

[2022-11-23 15:35:45] Features: 1/5 -- score: 0.9996242722408137[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  52 out of  52 | elapsed:   46.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s finished

[2022-11-23 15:36:32] Features: 2/5 -- score: 0.9996574557060341[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  51 out of  51 | elapsed:   52.0s finished
[Parallel(n_jobs=1)]: Using